# EddyPro API

This notebook gives an outline for working with the API.

* First step is to define your input primary parameters.  You need to give a siteID, everything else is *optional*.

    * kwargs (dict of all optional settings):\
        * sourceDir: if not provided, assumes data are already in Outputs folder
        * dateRange (if not provided, will run all files in the current year)
        * fileType (defaults to .ghg) must specify otherwise for 
        * metadataTemplate (only required for .dat files)



In [1]:
# def makeUserConfig()


# Example run for BB on one day in 2024
siteID = 'BB'
kwargs = {
    'dateRange':['2024-04-01','2024-04-02'],
    'sourceDir':"Y:/BB/raw/2024/",
    # 'reset':True,
    'metaDataUpdates':'Y:/BB/Manual_Metadata_Updates.csv',
    'priority':'High Priority'
}

# # # Example run for All BBS data from the Datadump Folder 20240516
# siteID = 'BBS'
# kwargs = {
#     'dateRange':['2023-06-01','2024-05-31'],
#     'sourceDir':"X:/BBS/EC_Station/2024/20240516/",
#     'searchTag':"BBS.FLUX",
#     'timeShift':30,
#     'fileType':'dat',
#     'metaDataTemplate':'Y:/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata',
# }


# # Cospectral correction options
# hf_meth = {
#     'Moncrieff et al. (1997)':'1',
#     'Horst (1997)':'2',
#     'Ibrom et al. (2007)':'3',
#     'Fratini et al. (2012)':'4', # Recommended for closed path systems, when sufficient data (>1 month) are available
#     'Massman (2000, 2001)':'5'
# }

# if siteID == 'BBS':
#     userDefinedEddyProSettings = {
#         'Project':{
#             'hf_meth':'1'
#             },
#         'RawProcess_Settings':{
#             'v_offset':0.08,
#             'u_offset':-0.04
#             },
#         }
#     kwargs['userDefinedEddyProSettings'] = userDefinedEddyProSettings

# if 'dateRange' not in kwargs:
#     kwargs['dateRange'] = None


# Check the configurations

In [2]:
import eddyProAPI
import importlib
importlib.reload(eddyProAPI)
# kwargs['siteID'] = siteID
kwargs = {
    'siteID':'BB',
    # 'dateRange':['2024-04-01','2024-04-02'],
    # 'sourceDir':"Y:/BB/raw/2024/",
    # 'reset':True,
    'metaDataUpdates':'Y:/BB/Manual_Metadata_Updates.csv',
    'priority':'High Priority'
}
kwargs['userDefinedEddyProSettings'] = {
        'Project':{
            'hf_meth':'4'
            }}

eP = eddyProAPI.eddyProAPI(**kwargs)
eP.config['BiometUser']

{'Biomet.Net': 'C:/Users/labuser/Biomet.net/', 'Database': 'W:/'}

## Run PreProcessing

In [3]:
pre = eddyProAPI.preProcessing(**kwargs)

Searching C:/Users/labuser/Temp//BB/raw/2024
[████████████████████████████████████████████████████████████] 1/1

Searching C:/Users/labuser/Temp//BB/raw/2024\01
[████████████████████████████████████████████████████████████] 1275/1275

Searching C:/Users/labuser/Temp//BB/raw/2024\02
[████████████████████████████████████████████████████████████] 1226/1226

Searching C:/Users/labuser/Temp//BB/raw/2024\03
[████████████████████████████████████████████████████████████] 1489/1489

Searching C:/Users/labuser/Temp//BB/raw/2024\04
[████████████████████████████████████████████████████████████] 1437/1437

Searching C:/Users/labuser/Temp//BB/raw/2024\05
[████████████████████████████████████████████████████████████] 1488/1488

Searching C:/Users/labuser/Temp//BB/raw/2024\06
[████████████████████████████████████████████████████████████] 404/404

Searching Y:/BB/raw/2024/01
[████████████████████████████████████████████████████████████] 1282/1282

Searching Y:/BB/raw/2024/02
[██████████████████████████

TypeError: '>=' not supported between instances of 'numpy.ndarray' and 'Timestamp'

# Run EddyPro in Parallel over date range

In [ ]:

import eddyProAPI
import importlib
importlib.reload(eddyProAPI)

kwargs['dumpToBiometDatabase']=True
eP = eddyProAPI.runEP(**kwargs)

fileInventory {'parse_dates': [0], 'index_col': [0], 'filepath_or_buffer': 'C:/Users/labuser/Temp//BB/metadata/fileInventory.csv'}


c:\Users\labuser\EddyPro_API\.venv\Lib\site-packages\pandas\io\parsers\c_parser_wrapper.py:234: RuntimeWarning: invalid value encountered in cast
  chunks = self._reader.read_low_memory(nrows)


ValueError: cannot safely convert passed user dtype of int32 for float64 dtyped data in column 5

In [ ]:

from collections import Counter, defaultdict
import pandas as pd
dtypes = defaultdict(lambda:'object',groupID='int')
pd.read_csv("C:\\Users\\labuser\\Temp\BB\\metadata/fileInventory.csv",dtype=dtypes)

c:\Users\labuser\EddyPro_API\.venv\Lib\site-packages\pandas\io\parsers\c_parser_wrapper.py:234: RuntimeWarning: invalid value encountered in cast
  chunks = self._reader.read_low_memory(nrows)


ValueError: cannot safely convert passed user dtype of int32 for float64 dtyped data in column 5